# Medium

## Subsets

* https://leetcode.com/problems/subsets/
***
* Time Complexity: O(n * 2$^{n}$)
    - reason being, there are 2^${n}$ subsets that can be made from n elements, and for each of those we must actually create the subset and push it into the res
    - if we just wanted to find the numbers, this would be O(1) since we can do simple math/bit manipulation
* Space Complexity: O(2$^{n}$)
    - going to need an array to store all of those subsets
***
* general backtracking for these types of problems:
    - create an array
    - and then you have 2 options:
        1. INCLUDE THE ELEMENT
        2. DON'T INCLUDE THE ELEMENT
    - if you include the element, just push it into the array
    - if you don't include it, then pop the element you just pushed from that array

In [2]:
/**
 * @param {number[]} nums
 * @return {number[][]}
 */

// recursive backtracking
var subsets = function(nums) {
    let res = [];
    
    const traverse = (subset, i) => {
        if (i >= nums.length) {
            // must create a copy of this b/c
            // the code that pushes/pops STILL AFFECTS THE ARRAY
            // subset isn't just the array, it's a reference to that array
            // so that's why, when we don't create a copy, we return O(n-squared) empty arrays
            res.push([...subset]);
            return;
        }
        
        // include the current element
        subset.push(nums[i]);
        traverse(subset, i + 1);
        
        // don't include the current element
        subset.pop(nums[i]);
        traverse(subset, i + 1);
    }
    
    traverse([], 0);
    return res;
};

// bottom-up iterative solution
var subsets = function(nums) {
    let res = [[]];
    
    for (let i = 0; i < nums.length; i++) {
        let resLen = res.length;
        for (let j = 0; j < resLen; j++) {
            res.push(res[j].concat(nums[i]));
        }
    }
    
    return res;
}

// bit manipulation
var subsets = function(nums) {
    let res = [];
    // counts from 0 ... nearest power
    // so if 1 << 3 = 8
    // reason we do this is b/c there are O(2 ^ n) subsets
    // so we use the binary sequence of the number as flags
    // for example: if nums = [1, 2, 3] and i = 5 (101),
    // then subset = [1, 3], b/c if the bit is 1, we include and if the bit is 0, we don't
    let len = 1 << nums.length;
    
    for (let i = 0; i < len; i++) {
        let subset = [];
        for (let j = 0; j < nums.length; j++) {
            if ((i >> j) & 1) {
                subset.push(nums[j]);
            }
        }
        res.push(subset);
    }
    
    return res;
}

## Combination Sum

* https://leetcode.com/problems/combination-sum/
***
* Time Complexity: O(2$^{t}$)
    - t = the target sum we are trying to get
        * reason being, we make 2 decisions for every element: include it or don't include it
    - and for each of these branches of the decision tree, it'll be as long as the target.
        * reason being, if arr = [1] and target = 3, our decision tree will go down at most 3 levels
* Space Complexity: O(n)
    - we're keeping an array, subset, with at most n elements for each of these function calls
    - and the number of function calls is at most O(t) b/c the longest branch in the recursion tree will be equal to the target
***
* we have 2 base cases:
    - if the current sum === target, then push it into the res array and return from it
    - if the current sum > target, then return b/c it's not a valid combination
* then we start at some index and loop through that index while adding the current element at index to the subset and update the sum, and we also should keep track of i
    - the usual trick is to push the element into the current subset --> call the function with that subset --> pop that element from the subset
        * why do we do this? 
            - since we are in a for-loop, if we add the element into the array WITHOUT popping it after we're doing traversing it, the subset will still have that element
            - instead, we should pop that element off so that we can refresh the subset to what it was in the function
    - in my solution below, i use the spread syntax (...) to achieve this same thing
    - so what this does is create a shallow copy of the array and adds the current element to it
        * the subset, if it doesn't have any nested objects or anything, will not be changed
        * so we essentially did subset.push(candidates[i]) and subset.pop() with just the spread syntax

In [1]:
/**
 * @param {number[]} candidates
 * @param {number} target
 * @return {number[][]}
 */

var combinationSum = function(candidates, target) {
    const res = [];

    const traverse = (arr, index, sum) => {
        if (sum > target) return;
        if (sum === target) {
            // much more efficient to create a copy of the array
            // once we reach the target
            // than to create a copy for every recursive call
            res.push([...arr]);
            return;
        }

        // we need to keep track of the index as well b/c
        // if it always just starts at 0, it'll just keep adding candidates[0] to it
        // and it'll never end
        // by doing this, once you meet the base cases and return
        // your index will move up by 1
        for (let i = index; i < candidates.length; i++) {
            // we include the item
            // and we traverse on it
            arr.push(candidates[i]);
            sum += candidates[i];
            traverse(arr, i, sum);
            
            // then we exclude the item to return it to its previous state
            // this allows for backtracking to work since the
            // array is a reference type and that reference is passed down the branch
            arr.pop();
            sum -= candidates[i];
        }
    }

    traverse([], 0, 0);

    return res;
};

## Permutations

* https://leetcode.com/problems/permutations/
***
* Time Complexity: O(n * n!)
    - reason being, there are n! permutations for n elements
    - and for each of these permutations, we also must create them as well since they want the list of it
* Space Complexity: O(n$^{2}$)
    - the branch of the recursion tree goes no deeper than n. reason being, once our permutation array reaches n in length, it'll immediately return from it
    - there are subsets of size n that we keep track of as well as we recurse
***
* there are 2 ways of doing this:
    1. the main way is by checking if we've already seen the element in the perm array we're tracking
        - if the element is already in there, then don't recurse on it
        - else, push element into perm array --> make the recursive call --> pop the element from the perm array once the recursive call returns
    2. the way I'm doing it is by create a subarray of the original array. this is so that we don't have to keep checking if the element is in the perm array b/c it'll be removed already
        - you can do this by doing [...subarray.slice(0,i), ...subarray.slice(i + 1)]
        - this will create a new array with elements from [0:i - 1] and from [i + 1: n]
        - my way is a bit faster b/c as we recurse down a branch, the subarray gets smaller and the slice operation gets a bit more efficient, whereas in the main way, the size of the array they have to check is always going to be equal to n no matter where they are in the recursion

In [5]:
/**
 * @param {number[]} nums
 * @return {number[][]}
 */

// about 10% faster since slice will progressively get more efficient
var permute = function(nums) {
    let res = [];
    
    const traverse = (perm, subNum) => {
        if (perm.length === nums.length) {
            res.push([...perm]);
            return;
        }
        
        for (let i = 0; i < subNum.length; i++) {
            perm.push(nums[i]);
            traverse(perm, [...subNum.slice(0, i), ...subNum.slice(i + 1)]);
            perm.pop();
        }
    }
    
    traverse([], nums);
    return res;
};

// this structure applies to more problems
var permute = function(nums) {
    let res = [];
    
    const traverse = (perm) => {
        if (perm.length === nums.length) {
            res.push([...perm]);
            return;
        }
        
        for (let i = 0; i < nums.length; i++) {
            if (perm.indexOf(nums[i]) === -1) {
                perm.push(nums[i]);
                traverse([...perm, nums[i]]);
                perm.pop();
            }
        }
    }
    
    traverse([]);
    
    return res;
}

## Subsets II

* https://leetcode.com/problems/subsets-ii/description/
***
* Time Complexity: O($2^{n}$)
    - a power set has $2^{n}$ subsets and we essentially go through $2^{n}$ calls to generate them all
* Space Complexity: O($2^{n}$)
    - have to store all $2^{n}$ subsets to return them
***
* similar to how you create subsets except you have a constraint you must adhere to
* in this case, the nums array contains duplicate values
    - you must sort it first
    - then when you've reached a duplicate, you can ignore it and move on until you reach a unique value
* __the reason why the condition is *i > index* instead of *i > 0* is because this would allow you to add a duplicate value ONCE when going down a branch naturally and not when you've backtracked__
    - for example: [1, 2, 2]
        * if i > 0 for the condition then when you go from [1] -> [1,2] -> ?, you will not create a [1,2,2] subset even though it's valid
        * this is because the recursive call of traverse([1,2], 2) will make that condition true
        * however, if i > index, then if i = 2, index = 2, and not greater than it
    - also, this will not create duplicate subsets because the algorithm will advance the index in the for loop and will either not run or it will enter into the conditional

In [1]:
/**
 * @param {number[]} nums
 * @return {number[][]}
 */
var subsetsWithDup = function(nums) {
    const res = [];
    nums.sort((a, b) => a - b);
    const traverse = (arr, index) => {
        for (let i = index; i < nums.length; i++) {
            if (i > index && (nums[i] === nums[i - 1])) continue;
            arr.push(nums[i]);
            traverse(arr, i + 1);
            arr.pop();
        }

        res.push([...arr]);
    }

    traverse([], 0);

    return res;
};